In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/캡스톤/기후평균커피가격통합데이터.csv")

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
X = df[df.columns[7:]]
y = df['Coffee_Price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
X_val.columns = X_val.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val)

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[62]	valid_0's l1: 11.0501


In [ ]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importance()
}).sort_values(by='importance', ascending=False)

In [ ]:
importance_df[:7]

,feature,importance
0,brazil_varginha_T2M_harvest_mean,417
42,colombia_manizales_T2M_harvest_mean,203
21,ethiopia_limu_T2M_harvest_mean,162
1,brazil_varginha_WS2M_harvest_mean,71
43,colombia_manizales_WS2M_harvest_mean,70
11,brazil_carmo_de_minas_PRECTOTCORR_harvest_mean,51
27,ethiopia_limu_PS_harvest_mean,45


In [ ]:
top_features = []
top_features.append('Date')
top_features.append('Coffee_Price')

In [ ]:
top_features += importance_df['feature'][:7].tolist()

In [ ]:
top_features

['Date',
 'Coffee_Price',
 'brazil_varginha_T2M_harvest_mean',
 'colombia_manizales_T2M_harvest_mean',
 'ethiopia_limu_T2M_harvest_mean',
 'brazil_varginha_WS2M_harvest_mean',
 'colombia_manizales_WS2M_harvest_mean',
 'brazil_carmo_de_minas_PRECTOTCORR_harvest_mean',
 'ethiopia_limu_PS_harvest_mean']

In [ ]:
df_selected = df[top_features]

In [ ]:
df_selected.to_csv('기후데이터피쳐선택.csv', index=False)